In [1]:
import pandas as pd
import os

In [2]:
BATTERY_DATA_FILE = "battery-state-reports-last-30-days.csv"
SUMMARY_OUTPUT_FILE = "summary.csv"

In [5]:
def get_directory(environment_name):
    try:
        directory = os.environ[environment_name] # i.e. Losant generated
    except:
        directory = os.getcwd() # local current directory
    return directory

def get_input_directory():
    return get_directory('INPUT_DIR')

def get_output_directory():
    return get_directory('OUTPUT_DIR')

def load_data():
    directory = get_input_directory()
    return pd.read_csv(os.path.join(directory, BATTERY_DATA_FILE))

def is_row_before_charging(row):
    return (row.battery_loss_next_row < 0) and (row.battery_loss_this_row > 0)

def remove_consecutive_same_values(df):
    df['is_new_value'] = df.groupby(['ID'])['battery'].shift(1) != df['battery']
    return df.loc[df['is_new_value']]

def get_rows_before_charging(df):
    # sorting not necessary, but is much easier to follow during dev
    # df.sort_values(by=['ID','Timestamp'],inplace=True)
    
    df.loc[:] = remove_consecutive_same_values(df)
    df['battery_loss_this_row'] = df.groupby(['ID'])['battery'].shift(1) - df['battery']
    df['battery_loss_next_row'] = df['battery'] - df.groupby(['ID'])['battery'].shift(-1)
    df.loc[:, 'is_low_before_charge'] = df.apply(is_row_before_charging, axis=1)
    return df[df['is_low_before_charge']]

def export_summary(summary_series, file_name=SUMMARY_OUTPUT_FILE):
    directory = get_output_directory()
    output = pd.DataFrame(summary_series).transpose()
    output.to_csv(os.path.join(directory, file_name),index=False)


In [6]:
df = load_data()
rows_before_charging = get_rows_before_charging(df)
export_summary(df['battery'].describe())